<style>
.analysis-title {
    color: #2563eb !important;
    font-size: 2.8rem;
    font-weight: 700;
    text-align: center;
    border-bottom: 4px solid #dbeafe;
    padding-bottom: 15px;
    margin-bottom: 25px;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
}
.metadata-box {
    background: linear-gradient(135deg, #f8fafc 0%, #e2e8f0 100%);
    border-left: 5px solid #3b82f6;
    padding: 20px;
    border-radius: 10px;
    margin: 20px 0;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
}
.metadata-text {
    color: #1e293b;
    font-size: 1.1rem;
    line-height: 1.8;
    margin: 0;
}
.overview-header {
    color: #1d4ed8 !important;
    font-size: 2rem;
    font-weight: 600;
    border-left: 5px solid #3b82f6;
    padding-left: 15px;
    margin-top: 30px;
    margin-bottom: 15px;
}
.section-text {
    color: #374151;
    font-size: 1.05rem;
    line-height: 1.7;
    text-align: justify;
}
.subsection-header {
    color: #4338ca !important;
    font-size: 1.4rem;
    font-weight: 600;
    margin-top: 25px;
    margin-bottom: 10px;
    border-bottom: 2px solid #e0e7ff;
    padding-bottom: 5px;
}
.data-list {
    background-color: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 8px;
    padding: 15px;
    margin: 15px 0;
}
.data-list ul {
    margin: 0;
    color: #475569;
    font-size: 1rem;
}
.data-list li {
    margin-bottom: 8px;
    padding-left: 5px;
}
.data-list code {
    background-color: #f1f5f9;
    color: #dc2626;
    padding: 2px 6px;
    border-radius: 4px;
    font-weight: 500;
}
</style>

<h1 class="analysis-title">Cell lines with Damaging Mutations</h1>

<div class="metadata-box">
<p class="metadata-text">
<strong style="color: #1e40af;">Project:</strong> Computational Biology DMV Petri Dish<br>
<strong style="color: #1e40af;">Author:</strong> Chris Indorf<br>
<strong style="color: #1e40af;">Date:</strong> August 12, 2025<br>
<strong style="color: #1e40af;">Language:</strong> Python
</p>
</div>

<h2 class="overview-header">Overview</h2>

<p class="section-text">This notebook performs data retrieval and visualization for analysis of lung cancer cell lines with damaging mutations. The analysis connects to a PostgreSQL data warehouse to query mutation data and creates an interactive histogram visualization.</p>

<h3 class="subsection-header">Data Sources</h3>
<div class="data-list">
<ul>
<li><strong>Database:</strong> data_warehouse (PostgreSQL)</li>
<li><strong>Tables:</strong>
    <ul>
        <li><code>im_dep_sprime_damaging_mutations</code></li>
        <li><code>im_dep_raw_secondary_dose_curve</code></li>
        </ul>
</li>
</ul>
</div>

<h3 class="subsection-header">Libraries Used</h3>
<div class="data-list">
<ul>
<li><code>psycopg2</code> - PostgreSQL database connectivity</li>
<li><code>dotenv</code> - To retrieve environmental variable</li>
<li><code>os</code> - To retrieve environmental variable</li>
<li><code>pandas</code> - Data manipulation and analysis</li>
<li><code>altair</code> - Statistical data visualization</li>
<li><code>numpy</code> - Numerical functions</li>
<li><code>warnings</code> - Suppresses warning messages</li>
</ul>
</div>

<h3 class="subsection-header">Change Log</h3>
<div class="data-list">
<ul>
<li><code>August 12, 2025 </code> - Version 1</li>
</ul>
</div>

In [1]:
# Import required libraries
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
import altair as alt
import numpy as np
import warnings
# Ignore warnings
warnings.filterwarnings('ignore')

<style>
.section-header {
    color: #1d4ed8 !important;
    font-size: 2rem;
    font-weight: 600;
    border-left: 5px solid #3b82f6;
    padding-left: 15px;
    margin-top: 30px;
    margin-bottom: 15px;
    background: #f8fafc;
    padding-top: 10px;
    padding-bottom: 10px;
}
.description-text {
    color: #374151;
    font-size: 1.05rem;
    line-height: 1.7;
    margin-bottom: 20px;
}
.process-box {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
}
.process-list {
    color: #475569;
    font-size: 1rem;
    margin: 0;
}
.process-list li {
    margin-bottom: 10px;
    font-weight: 500;
}
.code-highlight {
    background-color: #f8fafc;
    color: #dc2626;
    padding: 2px 6px;
    border-radius: 4px;
    font-family: 'Monaco', 'Consolas', monospace;
    font-weight: 600;
}
</style>

<h2 class="section-header">Data Retrieval</h2>

<p class="description-text">Connect to the PostgreSQL database and execute a query to retrieve lung cancer cell lines with damaging mutations. The query performs the following operations:</p>

<div class="process-box">
<ol class="process-list">
<li><strong>Filters</strong> for <span class="code-highlight">mutation_value = 2</span> (damaging mutations)</li>
<li><strong>Restricts</strong> cell lines to lung cancer cell lines (<span class="code-highlight">CCLE_name LIKE '%LUNG'</span>)</li>
<li><strong>Groups</strong> by <span class="code-highlight">cell line</span> and sums mutation values for each cell line group</li>
<li><strong>Orders</strong> results by cell line group</li>
</ol>
</div>

In [2]:
# Retrieve password from .env file  
load_dotenv()
password = os.getenv('DB_PASSWORD')  

# Establish database connection
conn = psycopg2.connect(
    host='dmvpetridishdatastore.dev',
    port=5432,
    database='data_warehouse',
    user='comp_bio_u2',
    password=password # password from .env
    # password='ENTER PASSWORD HERE' ### IN THIS DEVELOPMENT VERSION CAN REPLACE 'ENTER PASSWORD HERE' WITH ACTUAL PASSWORD
)

# Define SQL query to sum damaging mutations in lung tissue cell lines
query = """
SELECT cell_line, SUM(mutation_value) 
FROM public.im_dep_sprime_damaging_mutations
WHERE mutation_value = 2 
  AND cell_line IN (
    SELECT depmap_id 
    FROM public.im_dep_raw_secondary_dose_curve
    WHERE ccle_name LIKE '%LUNG')  
GROUP BY cell_line
ORDER BY cell_line;
"""

# Execute query and create DataFrame
df = pd.read_sql_query(query, conn)

# Close database connection
conn.close()

# Display results
print("Cell line damaging mutations:")
print(df)


Cell line damaging mutations:
     cell_line  sum
0   ACH-000012    2
1   ACH-000015   12
2   ACH-000021   30
3   ACH-000030    4
4   ACH-000035    4
..         ...  ...
88  ACH-000921   94
89  ACH-000924   68
90  ACH-000929   88
91  ACH-000945   30
92  ACH-001113   16

[93 rows x 2 columns]


<style>
.filtering-header {
    color: #1d4ed8 !important;
    font-size: 2rem;
    font-weight: 600;
    border-left: 5px solid #3b82f6;
    padding-left: 15px;
    margin-top: 30px;
    margin-bottom: 15px;
    background: #f8fafc;
    padding-top: 10px;
    padding-bottom: 10px;
}
.filtering-explanation {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
    color: #374151;
    font-size: 1.05rem;
    line-height: 1.7;
}
.highlight-number {
    background-color: #f8fafc;
    color: #475569;
    padding: 3px 8px;
    border-radius: 4px;
    font-weight: 700;
    font-size: 1.1rem;
}
</style>

<h2 class="filtering-header">Data Analysis</h2>

<div class="filtering-explanation">
<p>Basic analysis and display of the cell line dataframe to support verification of the resulting histogram</p>
</div>

In [4]:
# Analyze results of database retrieval Use pandas.
print ('Number of cell lines:', df.cell_line.count(), 
       'Smallest cell line number:', df.cell_line.min(), 
       'Largest cell_line number:', df.cell_line.max(), 
       'Total damaging mutations:', sum(df['sum']))

# Show distribution of mutation counts across cell lines
# Left column: number of cell lines with damaging mutations
# Right column: number of cell lines with that count
print('\n')
print(df['sum'].value_counts())

print(df.head(50))
print(df.tail(50))


Number of cell lines: 93 Smallest cell line number: ACH-000012 Largest cell_line number: ACH-001113 Total damaging mutations: 2334


sum
12    8
4     8
16    8
18    7
24    7
28    6
8     6
10    4
14    4
30    4
2     3
68    3
32    2
20    2
22    2
46    2
38    2
50    2
76    1
6     1
52    1
40    1
42    1
44    1
26    1
58    1
78    1
48    1
74    1
94    1
88    1
Name: count, dtype: int64
     cell_line  sum
0   ACH-000012    2
1   ACH-000015   12
2   ACH-000021   30
3   ACH-000030    4
4   ACH-000035    4
5   ACH-000062    8
6   ACH-000066    4
7   ACH-000161    2
8   ACH-000176   32
9   ACH-000186   28
10  ACH-000261   28
11  ACH-000264    8
12  ACH-000292   16
13  ACH-000309   28
14  ACH-000311    4
15  ACH-000343   28
16  ACH-000367    4
17  ACH-000390   14
18  ACH-000395   14
19  ACH-000414    4
20  ACH-000416   76
21  ACH-000434   18
22  ACH-000444    8
23  ACH-000447    8
24  ACH-000454   10
25  ACH-000463   16
26  ACH-000481   16
27  ACH-000482   10
28  ACH-0

<style>
.viz-header {
    color: #1d4ed8 !important;
    font-size: 2rem;
    font-weight: 600;
    border-left: 5px solid #3b82f6;
    padding-left: 15px;
    margin-top: 30px;
    margin-bottom: 15px;
    background: #f8fafc;
    padding-top: 10px;
    padding-bottom: 10px;
}
.viz-description {
    color: #374151;
    font-size: 1.05rem;
    line-height: 1.7;
    margin-bottom: 20px;
}
.features-box {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
}
.features-title {
    color: #374151;
    font-size: 1.3rem;
    font-weight: 600;
    margin-top: 0;
    margin-bottom: 15px;
}
.features-list {
    color: #475569;
    font-size: 1rem;
    margin: 0;
}
.features-list li {
    margin-bottom: 12px;
    font-weight: 500;
}
.feature-highlight {
    background-color: #f8fafc;
    color: #374151;
    padding: 2px 6px;
    border-radius: 4px;
    font-weight: 600;
}
</style>

<h2 class="viz-header">Data Visualization</h2>

<p class="viz-description">Creates an interactive histogram using Altair to visualize a frequency distribution of damaging mutations in cell lines associated with lung cancer.</p>

<div class="features-box">
<h3 class="features-title">Chart Features:</h3>
<ul class="features-list">
<li><span class="feature-highlight">Slider widget:</span> Dynamic slider allows the viewer to interactively select the number of cell line bins</li>
<li><span class="feature-highlight">Tool tips:</span> Hovering mouse cursor over bin shows bin details</li>
</ul>
</div>

In [6]:
# Creates an interactive histogram with a slider to control the number of bins.

#  alt.Chart: Interactive Altair chart with working slider  
         
# Prepare data
df['cell_line_last_6'] = df['cell_line'].str[-6:]
df['cell_line_q'] = pd.to_numeric(df['cell_line_last_6'])
field = 'cell_line_q'
sum_field = 'sum'   

# Configure slider parameters
bin_param = alt.param(
    name='num_bins',  # Same name as working version
    value=20,
    bind=alt.binding_range(min=10, max=30, step=1, name='Number of Bins: ')
)

# Pre-aggregate all possible bin counts
all_binned_data = []

for n_bins in range(10, 31):
    df_temp = df.copy()
    df_temp['num_bins'] = n_bins  # Same field name as working
    
    # Create bin edges and assign data to bins
    bin_edges = np.linspace(df[field].min(), df[field].max(), n_bins + 1)
    df_temp['bin_assignment'] = pd.cut(df_temp[field], bins=bin_edges, include_lowest=True)
    
    
    # Group by bins and aggregate
    binned = df_temp.groupby(['bin_assignment', 'num_bins']).agg({
        sum_field: 'sum',
        field: 'count'
    }).reset_index()
    
    binned.columns = ['bin_assignment', 'num_bins', 'sum_value', 'count']
    
    
    # Calculate bin positions
    binned['bin_left'] = pd.to_numeric(binned['bin_assignment'].apply(lambda x: x.left))
    binned['bin_right'] = pd.to_numeric(binned['bin_assignment'].apply(lambda x: x.right))
    binned['bin_center'] = (binned['bin_left'] + binned['bin_right']) / 2

    binned = binned.drop('bin_assignment', axis=1)
    
    all_binned_data.append(binned)

# Combine all bin configurations
combined = pd.concat(all_binned_data, ignore_index=True)

# Create interactive chart 
chart = alt.Chart(combined).add_params(
    bin_param
).transform_filter(
    'datum.num_bins == num_bins'  # Exact same filter as working
).mark_bar(
    color='steelblue',
    stroke='white',
    strokeWidth=0.1  # Very thin for near-contiguous appearance
).encode(
    alt.X('bin_center:Q', 
            title='Cell line bin (binned by ACH number)',
            axis=alt.Axis(labelAngle=-90)),
    alt.Y('sum_value:Q', 
            title='Damaging mutations count',
            scale=alt.Scale(domain=[0, combined['sum_value'].max() * 1.1]))
).properties(
    width=600,
    height=400,
    title='Cell Lines with Damaging Mutations'
)

chart.show()    
 

alt.Chart(...)

<style>
.filtering-header {
    color: #1d4ed8 !important;
    font-size: 2rem;
    font-weight: 600;
    border-left: 5px solid #3b82f6;
    padding-left: 15px;
    margin-top: 30px;
    margin-bottom: 15px;
    background: #f8fafc;
    padding-top: 10px;
    padding-bottom: 10px;
}
.filtering-explanation {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
    color: #374151;
    font-size: 1.05rem;
    line-height: 1.7;
}
.highlight-number {
    background-color: #f8fafc;
    color: #475569;
    padding: 3px 8px;
    border-radius: 4px;
    font-weight: 700;
    font-size: 1.1rem;
}
</style>

<h2 class="filtering-header">Chart storage</h2>

<div class="filtering-explanation">
<p>Save charts to files in HTML and PDF formats and create a link to a shareable URL.</p>
</div>

In [7]:
# Save chart to HTML, PDF, and shareable URL. 
chart.save('Cell Lines with Damaging Mutations.html')
chart = chart.configure(background='white', padding=30).configure_view(fill='white')
chart.save('Cell Lines with Damaging Mutations.pdf')
chart.to_url() # Creates a shareable URL

'https://vega.github.io/editor/#/url/vega-lite/N4Igxg9gdgZglgcxALlANzgUwO4tJKAFzigFcJSBnAdTgBNCALFAZgAY2AacaYsiygAlMiRoVYdu8ADbSUIbIziFMIAL7cARgEMwAawQAnClDrzFy1dwAO2unRJJk7DSDrbC2vCCjaAtqrIbh7aALTaAOwArAAsAJwATNpsMGAAHGnaMDFRAIy5cQBsmpiFYGy5UZoRmGBx6tx+2oZ63oQAntaBIDqGINyQ0hB9QZQqmNKa0qRWIGPGet0WKv1zhAuYtAzMyGwAdLmumFCQDlBOoAAe3tqXcJTe0tol0gCC59KBoXFsrvATZiCmhIAH0wMcVH1uMRCJ95ABhCbSAAE0hImGRwKgyIAFFioJg6Jj2sjXvDBMiyH4SoYAJSrDpdeQAR1I2iIyg8cDQqlc7W8-2kgLmpD8ILQ2mms0oYElgVAdAgTRIKAA2lx8oU0nsEgBdVwwuFBAAi-m0CEcyL8pE8xGglGRkFIRAZnW6rPZMK5PPUrkYIgQYhQMUkIFshn8D2QqtAvgC8ipIKxDy0JEBoBI1ht8gj51mTWuzi4ID8KuQuWLce6ADlRTTkRAYMiAEIkSjIZGrMaYawoQ7cCVSlAJX666HKI0gRGyZEAGXRDuwykYyNNTQt52RAFkbVz7Qzc5QYMM-GrQDJIfJ3IRRXtE8nkQBeR+U0VJtvqMcKehMFCFUMACQyv6TTyGIhDWO2AD0UE8gg2h7BaTCkJoexwBAUHAZgTSwZg8GhGiKiwVEOr7LkewAFaUNAqzXtolCYIQUYKiE4TRPESQpOkmTZHkBTFKU5SVNUtT1NGsZvsmfbFpQb6DjMf4xAMJjiMgESplAIKfDAqn5HscQGRpIKGKIukJAkBxGeCRCYCMhQRHsbDxFEGgSWKUnljJcmSgpyCFAkynOqpLBGdpZkWbkRkmYGqkJCwFkBT0oLWZe5YRA5uTxAZ2U5

<style>
.results-header {
    color: #2563eb !important;
    font-size: 2.2rem;
    font-weight: 700;
    text-align: center;
    border: 3px solid #dbeafe;
    background: #f8fafc;
    padding: 20px;
    border-radius: 15px;
    margin: 30px 0 25px 0;
    text-shadow: 1px 1px 3px rgba(0,0,0,0.1);
}
.summary-intro {
    background: #f8fafc;
    color: black;
    padding: 25px;
    border-radius: 12px;
    margin: 25px 0;
    box-shadow: 0 8px 25px rgba(0,0,0,0.15);
}
.summary-text {
    font-size: 1.1rem;
    line-height: 1.7;
    margin: 0;
}
.highlight-stat {
    background-color: #f8fafc;
    color: #4338ca;
    padding: 4px 10px;
    border-radius: 6px;
    font-weight: 700;
    font-size: 1.15rem;
}
.key-findings {
    background-color: #f8fafc;
    border: 2px solid #dbeafe;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
}
.findings-title {
    color: #4338ca !important;
    font-size: 1.4rem;
    font-weight: 600;
    margin-top: 0;
    margin-bottom: 15px;
    border-bottom: 2px solid #dbeafe;
    padding-bottom: 8px;
}
.findings-list {
    color: #1e3a8a;
    font-size: 1rem;
    margin: 0;
}
.findings-list li {
    margin-bottom: 10px;
    padding-left: 8px;
    border-left: 3px solid #60a5fa;
    padding-top: 5px;
    padding-bottom: 5px;
    background-color: #f8fafc;
    margin-left: 0;
    padding-left: 12px;
}
.data-distribution {
    background: #f8fafc;
    border: 2px solid #dbeafe;
    border-radius: 10px;
    padding: 20px;
    margin: 20px 0;
}
.distribution-title {
    color: #4338ca !important;
    font-size: 1.4rem;
    font-weight: 600;
    margin-top: 0;
    margin-bottom: 15px;
}
.distribution-list {
    color: #1e3a8a;
    font-size: 1.1rem;
    font-weight: 500;
    margin: 0;
}
.distribution-list li {
    background-color: #f8fafc;
    margin-bottom: 8px;
    padding: 10px 15px;
    border-radius: 6px;
    border-left: 4px solid #60a5fa;
}
.conclusion-box {
    background: #f8fafc;
    border: 2px solid #dbeafe;
    border-radius: 12px;
    padding: 25px;
    margin: 25px 0;
    color: #1e3a8a;
    font-size: 1.05rem;
    line-height: 1.8;
    font-style: italic;
}
.emphasis-text {
    background-color: #f8fafc;
    color: #black;
    padding: 3px 8px;
    border-radius: 4px;
    font-weight: 600;
    font-style: normal;
}
</style>

<h2 class="results-header">Results Summary</h2>

<div class="summary-intro">
<p class="summary-text">The input data set used for this analysis (DepMap) shows that 93 lung cancer cell lines have damaging mutations in one or more genes. The total number of damaging mutations for these genes is 2334. The ACH numbers of these cell lines range from ACH-000012 to ACH-001113. Over half of these 2334 mutations show a strong clustering effect around ACH-000900. The significance of this clustering (if any) cannot be determined from the chart alone.</p>
</div>

